<a href="https://colab.research.google.com/github/Arthur-Barreto/Exoplanets/blob/main/RedeNeural/models_comparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook has the objective to compare SVM, Tree regresor and Random Forest model to classify transit palent model

In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## First, import the dataframe filtered, considering 64k data points on lightcurve.

In [ ]:
df_filtered = pd.read_pickle("/content/gdrive/MyDrive/logArthur/df_filtered.pkl")

In [ ]:
df_filtered.head()

,sky_position,max_power,period_at_max_power,transit_time_at_max_power,duration_at_max_power,has_flux,lc_flux,lc_flux_2
8,Kepler-9,27691.850489,19.276027602760276 d,143.898013,0.15 d,1,"[0.9975966720091157, 0.9973999922352278, 0.997...","[0.9975966720091157, 0.9973999922352278, 0.997..."
10,Kepler-11,3323.241291,45.373237323732376 d,148.472045,0.2 d,1,"[1.0001171265268693, 1.0000895883816208, 0.999...","[1.0001171265268693, 1.0000895883816208, 0.999..."
15,Kepler-16,150214.702840,359.41764176417644 d,352.263886,0.33 d,1,"[1.000646710395813, 1.000847578048706, 1.00074...","[1.000646710395813, 1.000847578048706, 1.00074..."
18,Kepler-19,2198.032117,65.00600060006 d,145.319322,0.15 d,1,"[1.0004145645415485, 1.0001728345558498, 1.000...","[1.0004145645415485, 1.0001728345558498, 1.000..."
19,Kepler-20,22398.128200,10.856285628562857 d,138.472644,0.33 d,1,"[0.9999672136385541, 0.9999018197503611, 0.999...","[0.9999672136385541, 0.9999018197503611, 0.999..."


## Now, check the number of 0 and 1, then filter do aplly our models

In [ ]:
zeros = df_filtered[df_filtered['has_flux'] == 0]
ones = df_filtered[df_filtered['has_flux'] == 1]

num_zeros = len(zeros)
num_ones = len(ones)

print(f'num_zeros= {num_zeros} | num_ones= {num_ones}\n')

num_zeros= 300 | num_ones= 248



In [ ]:
selected_zeros = zeros.sample(n=248)
selected_ones = ones.sample(n=248)

selected_data = pd.concat([selected_zeros, selected_ones])
selected_data = selected_data.reset_index()

selected_data.describe()

,index,max_power,transit_time_at_max_power,has_flux
count,496.000000,4.960000e+02,496.000000,496.000000
mean,505.171371,4.849017e+04,185.017420,0.500000
std,285.083147,2.807735e+05,79.747187,0.500505
min,8.000000,3.281983e+01,120.754127,0.000000
25%,251.500000,3.029960e+02,138.055518,0.000000
50%,506.000000,9.956547e+02,148.331701,0.500000
75%,754.000000,4.222253e+03,189.758429,1.000000
max,983.000000,4.499717e+06,509.292285,1.000000


## Now, it's just import the libraries to use the SVM, Tree Regressor and Random Forest

In [ ]:
# split data, randomly
from sklearn.model_selection import train_test_split
# necessary imports to run the models and compare
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
import pickle 

In [ ]:
with open('/content/gdrive/MyDrive/logArthur/log_models.csv', 'a') as f:
    f.write("accuracy_clf,accuracy_train_clf,accuracy_tr,accuracy_train_tr,accuracy_rf,accuracy_train_rf \n")

In [ ]:
for i in range(491,1001):
    X_train, X_test, y_train, y_test = train_test_split(list(selected_data.lc_flux_2), selected_data.has_flux, test_size=0.5, random_state=109) # 50% training and 50% test
    
    #Create a svm Classifier
    clf = svm.SVC(kernel='linear') # Linear Kernel
    #Train the model using the training sets
    clf.fit(X_train, y_train)
    #Predict the response for test dataset
    y_pred_clf = clf.predict(X_test)
    y_pred_train_clf = clf.predict(X_train)
    ## evaluating the accuracy
    accuracy_clf = accuracy_score(y_test, y_pred_clf)
    accuracy_train_clf = accuracy_score(y_test, y_pred_train_clf)

    # creat tree regressor classifier
    tr = DecisionTreeRegressor()
    tr.fit(X_train, y_train)
    y_pred_tr = tr.predict(X_test)
    y_pred_train_tr = tr.predict(X_train)
    ## evaluating the accuracy
    accuracy_tr = accuracy_score(y_test, y_pred_tr)
    accuracy_train_tr = accuracy_score(y_test, y_pred_train_tr)

    # creat random forest classifier
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    y_pred_train_rf = rf.predict(X_train)
    ## evaluating the accuracy
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    accuracy_train_rf = accuracy_score(y_test, y_pred_train_rf)

    with open('/content/gdrive/MyDrive/logArthur/log_models.csv', 'a') as f:

        #################### writing results on log file #######################
        f.write(f'{str(accuracy_clf)},{str(accuracy_train_clf)},{str(accuracy_tr)},{str(accuracy_train_tr)},{str(accuracy_rf)},{str(accuracy_train_rf)} \n')

In [ ]:
df_data_models = pd.read_csv("/content/gdrive/MyDrive/logArthur/log_models.csv")

In [ ]:
df_data_models.tail()

,accuracy_clf,accuracy_train_clf,accuracy_tr,accuracy_train_tr,accuracy_rf,accuracy_train_rf
996,0.524194,0.504032,0.548387,0.483871,0.689516,0.483871
997,0.524194,0.504032,0.600806,0.483871,0.685484,0.483871
998,0.524194,0.504032,0.588710,0.483871,0.705645,0.483871
999,0.524194,0.504032,0.568548,0.483871,0.717742,0.483871
1000,0.524194,0.504032,0.596774,0.483871,0.713710,0.483871


In [ ]:
df_data_models.describe()

,accuracy_clf,accuracy_train_clf,accuracy_tr,accuracy_train_tr,accuracy_rf,accuracy_train_rf
count,1001.000000,1001.000000,1001.000000,1.001000e+03,1001.000000,1.001000e+03
mean,0.516282,0.504032,0.581209,4.838710e-01,0.702813,4.838710e-01
std,0.008067,0.000000,0.018182,1.666167e-16,0.015446,1.666167e-16
min,0.508065,0.504032,0.512097,4.838710e-01,0.645161,4.838710e-01
25%,0.508065,0.504032,0.568548,4.838710e-01,0.693548,4.838710e-01
50%,0.524194,0.504032,0.580645,4.838710e-01,0.701613,4.838710e-01
75%,0.524194,0.504032,0.592742,4.838710e-01,0.713710,4.838710e-01
max,0.524194,0.504032,0.637097,4.838710e-01,0.754032,4.838710e-01
